In [1]:
# define the vgg network and load pre-trained weights
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Dense, Dropout, Flatten, Lambda
from keras import optimizers
from keras import backend as K
K.set_image_data_format("channels_first")

def preprocess(img):
    """
    subtract average pixes of each channel
    and reverse the channel axies from 'rgb' to 'bgr'
    Args:
        img: (batch_size, channel_size, height, width)
    """
    vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))
    return (img - vgg_mean)[:, ::-1] # 注意第一个维度是batch_size

def AddConvBlock(model, layers, filters):
    """
    Args:
        model: keras model
        layers: number of padding + conv layers
        filters: number of filters
    """
    for _ in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
def AddFCBlock(model, units, dropout=0.5):
    """
    Args:
        model: keras sequential model
        units: positive integer, dimensionality of the output space
        dropout: dropout rate
    """
    model.add(Dense(units, activation='relu'))
    if dropout is not None:
        model.add(Dropout(dropout))
    

vgg_model = Sequential()
# 预处理：这里要指定输入张量的维度。在后面的模块中一般不需要考虑上一层的输入维度，keras会自动计算
vgg_model.add(Lambda(preprocess, input_shape=(3, 224, 224), output_shape=(3, 224, 224)))
# 添加卷积模块
AddConvBlock(vgg_model, 2, 64)
AddConvBlock(vgg_model, 2, 128)
AddConvBlock(vgg_model, 3, 256)
AddConvBlock(vgg_model, 3, 512)
AddConvBlock(vgg_model, 3, 512)
# 将(channels, height, width)的三维张量打平成(channels * height * width, )的一维张量
vgg_model.add(Flatten())
# 添加全连接层和dropout
AddFCBlock(vgg_model, units=4096, dropout=0.5)
AddFCBlock(vgg_model, units=4096, dropout=0.5)
# the last layer: softmax layer
vgg_model.add(Dense(units=1000, activation="softmax"))

vgg_model.load_weights("./models/vgg16.h5")

Using Theano backend.


In [2]:
# pop the fc layer and use flatten layer as output for feature extraction
while type(vgg_model.layers[-1]) is not Flatten:
    vgg_model.pop()

#vgg_model.compile(optimizer="sgd", loss="mse", metrics=["mae"])

In [32]:
# define the batch flow for images from directory
from keras.preprocessing.image import *
import os

# 自定义DirectoryIterator类，可以返回自定义的label
class CustomDirectoryIterator(DirectoryIterator):
    def __init__(self, directory, image_data_generator, output_data_generator, target_size, batch_size, shuffle):
        super(CustomDirectoryIterator, self).__init__(directory, image_data_generator,
                 target_size=target_size, color_mode='rgb',
                 classes=None, class_mode=None,
                 batch_size=batch_size, shuffle=shuffle, seed=None,
                 data_format=None,
                 save_to_dir=None, save_prefix='', save_format='png',
                 follow_links=False)
        self.output_data_generator = output_data_generator
    
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        batch_y = np.zeros(current_batch_size , dtype=K.floatx())
        grayscale = self.color_mode == 'grayscale'
        # build batch of image data
        for i, j in enumerate(index_array):
            fname = self.filenames[j]
            img = load_img(os.path.join(self.directory, fname),
                           grayscale=grayscale,
                           target_size=self.target_size)
            x = img_to_array(img, data_format=self.data_format)
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x
            batch_y[i] = self.output_data_generator(fname)
        return batch_x, batch_y

# 定义数据根目录（先用sample目标调试程序，真正训练时切换到data目录下）
path = "./data/sample/"

# 定义批处理的数据集大小：较小的batch_size可以增加权重调整的次数，同时节省内存的开销
batch_size = 16 

# 获取自定义label的lambda函数。注意传入的参数是图片文件名
ODG = lambda img_filename: os.path.splitext(img_filename)[0].split("_")[1]

# 图片预处理工具类
IDG = ImageDataGenerator()

# 从目录文件中流式读取数据，避免训练中一次性加载爆内存
train_batch = CustomDirectoryIterator(path + "train/", IDG, ODG, 
                                      target_size=(224, 224), batch_size=batch_size, shuffle=False)
valid_batch = CustomDirectoryIterator(path + "valid/", IDG, ODG,
                                      target_size=(224, 224), batch_size=batch_size, shuffle=False)

Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.


In [66]:
all_features = []
all_labels = []
steps = train_batch.samples // train_batch.batch_size
for _ in range(steps+1):
    img, label = train_batch.next()
    feature = vgg_model.predict_on_batch(img)
    all_features.extend(feature)
    all_labels.extend(label)
    
all_features = np.array(all_features)
all_labels = np.array(all_labels)

In [71]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

save_array(path+"results/train_feature.bc", all_features)
save_array(path+"results/train_label.bc", all_labels)

In [72]:
all_features = load_array(path+"results/train_feature.bc")